<a href="https://colab.research.google.com/github/suriarasai/BEAD2026/blob/main/colab/03b_DataMungingUsingFunctions_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Munging Examples
Data munging (or data wrangling) in PySpark for taxi booking services typically involves cleaning, transforming, and enriching raw data to make it suitable for analysis.
Here are some common data munging tasks with examples:

Install Spark
(ONLY IF SPARK DOESNT WORK)

In [ ]:
# install pyspark using pip
!pip install --ignore-install -q pyspark
# install findspark using pip
!pip install --ignore-install -q findspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 15.4 MB/s eta 0:00:00


## SparkSession
Start Spark Session

In [4]:
from pyspark.sql import SparkSession
# import collections
spark = SparkSession.builder.master("local").appName("My First PySpark App").getOrCreate()
sc = spark.sparkContext

## Drive
Connect to Google Drive

In [1]:
# to read in data from a text file, first upload the data file into your google drive and then mount your google drive onto colab
from google.colab import drive
# to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True)
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Taxi Data Munging

### Munging with RDD
RDDs are lower-level and require more manual handling, but they give you more control over the data processing.

Data munging using PySpark RDDs requires manual transformation functions. We'll work with tuples representing (trip_id, pickup_time, dropoff_time, dropoff_location, fare_amount)

### Creating Sample Data

In [5]:
# Sample Data as tuples
data = [
    (1, "2025-02-18 10:00:00", "2025-02-18 10:30:00", None, 15.5),
    (2, "2025-02-18 11:00:00", None, "Downtown", 20.0),
    (3, None, "2025-02-18 12:00:00", "Airport", None),
]

# Create RDD
rdd = sc.parallelize(data)

# Display the RDD
print("Original RDD:")
for row in rdd.collect():
    print(row)

Original RDD:
(1, '2025-02-18 10:00:00', '2025-02-18 10:30:00', None, 15.5)
(2, '2025-02-18 11:00:00', None, 'Downtown', 20.0)
(3, None, '2025-02-18 12:00:00', 'Airport', None)


## Handling Missing Values
Replace None values with default values using map transformation

In [6]:
def fill_missing_values(row):
    """
    Fill missing values in a row
    Row format: (trip_id, pickup_time, dropoff_time, dropoff_location, fare_amount)
    """
    trip_id, pickup_time, dropoff_time, dropoff_location, fare_amount = row

    # Replace None with default values
    dropoff_location = dropoff_location if dropoff_location is not None else "Unknown"
    fare_amount = fare_amount if fare_amount is not None else 0.0

    return (trip_id, pickup_time, dropoff_time, dropoff_location, fare_amount)

# Apply transformation
rdd_cleaned = rdd.map(fill_missing_values)

print("\nRDD after filling missing values:")
for row in rdd_cleaned.collect():
    print(row)


RDD after filling missing values:
(1, '2025-02-18 10:00:00', '2025-02-18 10:30:00', 'Unknown', 15.5)
(2, '2025-02-18 11:00:00', None, 'Downtown', 20.0)
(3, None, '2025-02-18 12:00:00', 'Airport', 0.0)


### Converting Datetime Formats
Parse string timestamps to datetime objects


In [7]:
def parse_timestamps(row):
    """
    Convert timestamp strings to datetime objects
    """
    trip_id, pickup_time, dropoff_time, dropoff_location, fare_amount = row

    # Parse timestamps
    if pickup_time:
        try:
            pickup_time = datetime.strptime(pickup_time, "%Y-%m-%d %H:%M:%S")
        except:
            pickup_time = None

    if dropoff_time:
        try:
            dropoff_time = datetime.strptime(dropoff_time, "%Y-%m-%d %H:%M:%S")
        except:
            dropoff_time = None

    return (trip_id, pickup_time, dropoff_time, dropoff_location, fare_amount)

# Apply transformation
rdd_with_timestamps = rdd.map(parse_timestamps)

print("\nRDD with parsed timestamps:")
for row in rdd_with_timestamps.collect():
    print(row)


RDD with parsed timestamps:
(1, None, None, None, 15.5)
(2, None, None, 'Downtown', 20.0)
(3, None, None, 'Airport', None)


### Filtering Invalid Data
Keep only valid trips with positive fares and non-null pickup times

In [8]:
def is_valid_trip(row):
    """
    Check if a trip is valid
    """
    trip_id, pickup_time, dropoff_time, dropoff_location, fare_amount = row

    # Valid if fare > 0 and pickup_time is not None
    return fare_amount is not None and fare_amount > 0 and pickup_time is not None

# Filter valid trips
rdd_valid = rdd_with_timestamps.filter(is_valid_trip)

print("\nValid trips:")
for row in rdd_valid.collect():
    print(row)


Valid trips:


### Feature Engineering - Trip Duration
Calculate trip duration in minutes

In [9]:
def calculate_trip_duration(row):
    """
    Calculate trip duration in minutes
    Returns: (trip_id, pickup_time, dropoff_time, dropoff_location, fare_amount, trip_duration_mins)
    """
    trip_id, pickup_time, dropoff_time, dropoff_location, fare_amount = row

    # Calculate duration if both times are available
    if pickup_time and dropoff_time:
        duration = (dropoff_time - pickup_time).total_seconds() / 60
    else:
        duration = None

    return (trip_id, pickup_time, dropoff_time, dropoff_location, fare_amount, duration)

# Add trip duration
rdd_with_duration = rdd_with_timestamps.map(calculate_trip_duration)

print("\nRDD with trip duration:")
for row in rdd_with_duration.collect():
    print(row)


RDD with trip duration:
(1, None, None, None, 15.5, None)
(2, None, None, 'Downtown', 20.0, None)
(3, None, None, 'Airport', None, None)


### Geospatial Filtering
Filter trips by dropoff location

In [10]:
def is_location(row, location):
    """
    Check if trip ended at specified location
    """
    trip_id, pickup_time, dropoff_time, dropoff_location, fare_amount, duration = row
    return dropoff_location == location

# Filter trips to Changi
rdd_changi = rdd_with_duration.filter(lambda row: is_location(row, "Changi"))

print("\nTrips to Changi:")
changi_trips = rdd_changi.collect()
if changi_trips:
    for row in changi_trips:
        print(row)
else:
    print("No trips found to Changi")

# Filter trips to Airport
rdd_airport = rdd_with_duration.filter(lambda row: is_location(row, "Airport"))
print("\nTrips to Airport:")
for row in rdd_airport.collect():
    print(row)


Trips to Changi:
No trips found to Changi

Trips to Airport:
(3, None, None, 'Airport', None, None)


### Aggregation - Daily Revenue
Calculate total fare per day using reduceByKey

In [11]:
def extract_date_and_fare(row):
    """
    Extract date and fare amount for aggregation
    Returns: (date_string, fare_amount)
    """
    trip_id, pickup_time, dropoff_time, dropoff_location, fare_amount, duration = row

    if pickup_time and fare_amount:
        date_str = pickup_time.strftime("%Y-%m-%d")
        return (date_str, fare_amount)
    return (None, 0.0)

# Create pair RDD and aggregate
rdd_daily_fare = rdd_with_duration \
    .map(extract_date_and_fare) \
    .filter(lambda x: x[0] is not None) \
    .reduceByKey(lambda a, b: a + b)

print("\nDaily total fare:")
for date, total_fare in rdd_daily_fare.collect():
    print(f"Date: {date}, Total Fare: ${total_fare:.2f}")


Daily total fare:


### Data Deduplication
Remove duplicate trips based on trip_id

In [12]:
# Method 1: Using distinct() on the entire row (works if rows are identical)
rdd_deduped = rdd_with_duration.distinct()

# Method 2: Remove duplicates based on trip_id (keeps first occurrence)
def get_trip_id_row_pair(row):
    """Create (trip_id, row) pair"""
    return (row[0], row)

rdd_deduped_by_id = rdd_with_duration \
    .map(get_trip_id_row_pair) \
    .reduceByKey(lambda a, b: a) \
    .map(lambda x: x[1])

print("\nDeduplicated trips:")
for row in rdd_deduped_by_id.collect():
    print(row)


Deduplicated trips:
(1, None, None, None, 15.5, None)
(2, None, None, 'Downtown', 20.0, None)
(3, None, None, 'Airport', None, None)


## Taxi Data Munging (DataFrames)
Data munging (or data wrangling) in PySpark for taxi booking services typically involves cleaning, transforming, and enriching raw data to make it suitable for analysis. Here are some common data munging tasks with examples:

### Handling Missing Values
Taxi booking datasets often contain missing values in fields like dropoff_location, fare_amount, etc.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("TaxiDataMunging").getOrCreate()

# Sample Data
data = [
    (1, "2025-02-18 10:00:00", "2025-02-18 10:30:00", None, 15.5),
    (2, "2025-02-18 11:00:00", None, "Downtown", 20.0),
    (3, None, "2025-02-18 12:00:00", "Airport", None),
]
columns = ["trip_id", "pickup_time", "dropoff_time", "dropoff_location", "fare_amount"]

df = spark.createDataFrame(data, columns)


# Fill missing values
In PySpark you can handle missing (null or NaN) values in a couple of ways:

1. Using the DataFrame’s fillna() Method
This method lets you replace missing values with a specified constant. You can either fill every column with the same value or pass a dictionary to specify different fill values per column.
2. Using the Imputer from pyspark.ml.feature
For numerical columns, you might prefer to impute missing values with a statistic such as the mean or median.

Both methods are commonly used depending on the nature of your data and the imputation strategy you need.

fillna() / na.fill(): Use these for simple replacements where a constant or specific value per column is appropriate.

Imputer: Best for numerical columns when you want a data-driven replacement (e.g., mean or median).

In [ ]:
df_cleaned = df.fillna({"dropoff_location": "Unknown", "fare_amount": 0.0})

df_cleaned.show()


+-------+-------------------+-------------------+----------------+-----------+
|trip_id|        pickup_time|       dropoff_time|dropoff_location|fare_amount|
+-------+-------------------+-------------------+----------------+-----------+
|      1|2025-02-18 10:00:00|2025-02-18 10:30:00|         Unknown|       15.5|
|      2|2025-02-18 11:00:00|               NULL|        Downtown|       20.0|
|      3|               NULL|2025-02-18 12:00:00|         Airport|        0.0|
+-------+-------------------+-------------------+----------------+-----------+



# Converting Datetime Formats
Timestamps are usually in string format. We convert them to PySpark Timestamp for easy calculations. Below example converts time fields from string to timestamp for further processing.


In [ ]:
from pyspark.sql.functions import to_timestamp

df = df.withColumn("pickup_time", to_timestamp(col("pickup_time")))
df = df.withColumn("dropoff_time", to_timestamp(col("dropoff_time")))

df.show()


+-------+-------------------+-------------------+----------------+-----------+
|trip_id|        pickup_time|       dropoff_time|dropoff_location|fare_amount|
+-------+-------------------+-------------------+----------------+-----------+
|      1|2025-02-18 10:00:00|2025-02-18 10:30:00|            NULL|       15.5|
|      2|2025-02-18 11:00:00|               NULL|        Downtown|       20.0|
|      3|               NULL|2025-02-18 12:00:00|         Airport|       NULL|
+-------+-------------------+-------------------+----------------+-----------+



# Filtering Invalid Data
Remove bookings with negative fares or missing crucial data. Below example keeps only valid trips where fare is positive and pickup time is not missing.


In [ ]:
df_valid = df.filter((col("fare_amount") > 0) & col("pickup_time").isNotNull())
df_valid.show()


+-------+-------------------+-------------------+----------------+-----------+
|trip_id|        pickup_time|       dropoff_time|dropoff_location|fare_amount|
+-------+-------------------+-------------------+----------------+-----------+
|      1|2025-02-18 10:00:00|2025-02-18 10:30:00|            NULL|       15.5|
|      2|2025-02-18 11:00:00|               NULL|        Downtown|       20.0|
+-------+-------------------+-------------------+----------------+-----------+



# Feature Engineering
This example focuses on Trip Duration. The computation is in minutes.

In [ ]:
from pyspark.sql.functions import unix_timestamp, col

df = df.withColumn("trip_duration_mins", \
                   (unix_timestamp(col("dropoff_time")) -  \
                    unix_timestamp(col("pickup_time"))) / 60)

df.show()


+-------+-------------------+-------------------+----------------+-----------+------------------+
|trip_id|        pickup_time|       dropoff_time|dropoff_location|fare_amount|trip_duration_mins|
+-------+-------------------+-------------------+----------------+-----------+------------------+
|      1|2025-02-18 10:00:00|2025-02-18 10:30:00|            NULL|       15.5|              30.0|
|      2|2025-02-18 11:00:00|               NULL|        Downtown|       20.0|              NULL|
|      3|               NULL|2025-02-18 12:00:00|         Airport|       NULL|              NULL|
+-------+-------------------+-------------------+----------------+-----------+------------------+



# Geospatial Filtering (Pickups in Specific Area)
Filter trips that started from a given region (e.g., Manhattan). Selects trips that ended in Changi.


In [ ]:
df_filtered = df.filter(col("dropoff_location") == "Changi")
df_filtered.show()


+-------+-----------+------------+----------------+-----------+------------------+
|trip_id|pickup_time|dropoff_time|dropoff_location|fare_amount|trip_duration_mins|
+-------+-----------+------------+----------------+-----------+------------------+
+-------+-----------+------------+----------------+-----------+------------------+



# Aggregation
Summarize daily revenue. Calculates total fare collected per day.

In [ ]:
from pyspark.sql.functions import date_format, sum

df_daily_fare = df.withColumn("date", date_format(col("pickup_time"), "yyyy-MM-dd")) \
                  .groupBy("date").agg(sum("fare_amount").alias("total_fare"))

df_daily_fare.show()


+----------+----------+
|      date|total_fare|
+----------+----------+
|2025-02-18|      35.5|
|      NULL|      NULL|
+----------+----------+



# Data Deduplication



In [ ]:
df_deduped = df.dropDuplicates(["trip_id"])
df_deduped.show()

+-------+-------------------+-------------------+----------------+-----------+------------------+
|trip_id|        pickup_time|       dropoff_time|dropoff_location|fare_amount|trip_duration_mins|
+-------+-------------------+-------------------+----------------+-----------+------------------+
|      1|2025-02-18 10:00:00|2025-02-18 10:30:00|            NULL|       15.5|              30.0|
|      2|2025-02-18 11:00:00|               NULL|        Downtown|       20.0|              NULL|
|      3|               NULL|2025-02-18 12:00:00|         Airport|       NULL|              NULL|
+-------+-------------------+-------------------+----------------+-----------+------------------+



# Bucketing Fare Amounts
Categorize trips based on fare price. Groups fares into Low, Medium, and High categories.


In [ ]:
from pyspark.sql.functions import when
df = df.withColumn("fare_category", when(col("fare_amount") < 10, "Low Fare") \
       .when((col("fare_amount") >= 10) & (col("fare_amount") < 30),  \
             "Medium Fare").otherwise("High Fare"))

df.show()


+-------+-------------------+-------------------+----------------+-----------+------------------+-------------+
|trip_id|        pickup_time|       dropoff_time|dropoff_location|fare_amount|trip_duration_mins|fare_category|
+-------+-------------------+-------------------+----------------+-----------+------------------+-------------+
|      1|2025-02-18 10:00:00|2025-02-18 10:30:00|            NULL|       15.5|              30.0|  Medium Fare|
|      2|2025-02-18 11:00:00|               NULL|        Downtown|       20.0|              NULL|  Medium Fare|
|      3|               NULL|2025-02-18 12:00:00|         Airport|       NULL|              NULL|    High Fare|
+-------+-------------------+-------------------+----------------+-----------+------------------+-------------+



# Join Data
Merge booking details with customer data.

In [ ]:
customer_data = [(1, "Alice"), (2, "Bob"), (3, "Charlie")]
customer_df = spark.createDataFrame(customer_data, ["customer_id", "customer_name"])

trip_data = [(1, 1, 20.5), (2, 2, 15.0), (3, 1, 30.0)]
trip_df = spark.createDataFrame(trip_data, ["trip_id", "customer_id", "fare_amount"])

df_joined = trip_df.join(customer_df, "customer_id", "inner")
df_joined.show()


+-----------+-------+-----------+-------------+
|customer_id|trip_id|fare_amount|customer_name|
+-----------+-------+-----------+-------------+
|          1|      1|       20.5|        Alice|
|          1|      3|       30.0|        Alice|
|          2|      2|       15.0|          Bob|
+-----------+-------+-----------+-------------+



# Save Data
Save the cleaned dataset as Parquet for efficient querying. Stores processed data in Parquet format.

In [ ]:
df_cleaned.write.mode("overwrite").parquet("output/taxi_cleaned.parquet")

# Final Thoughts
These PySpark data munging techniques help clean, filter, and transform raw taxi booking data, making it ready for analytics and reporting.
Would you like an end-to-end PySpark pipeline for this? 🚀

Yes, you can install Python and PyCharm on a Windows machine and run PySpark code within a Podman Kubernetes cluster.
